In [28]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from functools import reduce
import unicodedata
from pyspark.sql.functions import col

In [1]:
#iniciar spark

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1a2974d0-3be9-42ef-9af3-abd4c55810f0;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 569ms :: artifacts dl 26ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [24]:
dfRandon = spark.read.json("s3a://catarina-3sprint-sptech-bucket-bruto/randon.json", multiLine=True)

In [20]:
print(dfRandon) 

DataFrame[id: bigint, litros_acumulado: string, memory: bigint, ph: double, qnt_litros: bigint]


In [25]:
dfRandon.show()

+---+----------------+------+----+----------+
| id|litros_acumulado|memory|  ph|qnt_litros|
+---+----------------+------+----+----------+
|  1|      5000050000|     0|0.16|    100000|
|  2|     20000100000|    14|0.26|    200000|
|  3|     45000150000|    15|0.46|    300000|
|  4|     80000200000|    54|0.58|    400000|
|  5|        1,25E+11|    55|0.72|    500000|
|  6|      5000050000|    83|0.13|    100000|
|  7|     20000100000|    83|0.37|    200000|
|  8|     45000150000|    84|0.44|    300000|
|  9|     80000200000|    37|0.78|    400000|
| 10|        1,25E+11|    38|0.76|    500000|
| 11|      5000050000|    38|0.13|    100000|
| 12|     20000100000|    38|0.26|    200000|
| 13|     45000150000|    39|0.37|    300000|
| 14|     80000200000|    34|0.47|    400000|
| 15|        1,25E+11|    35|0.57|    500000|
| 16|          500500|    35|0.16|      1000|
| 17|          605550|    35|0.26|      1100|
| 18|          720600|    33|0.46|      1200|
| 19|          845650|    31|0.58|

In [34]:
colunas = dfRandon.columns
novas_colunas = list(map(lambda item: unicodedata.normalize('NFKD', item.replace("(", "").replace(")", "").replace("-","").replace(
    " ", "_").upper()).encode('ascii', 'ignore').decode('utf-8', 'ignore'), colunas))
dfRandonNovasColunas = reduce(lambda data, idx: data.withColumnRenamed(colunas[idx], novas_colunas[idx]),
                              range(len(colunas)), dfRandon)

In [35]:
dfRandonTratado = dfRandonNovasColunas.selectExpr("id as ID",
                                                           "litros_acumulado as LITROS_GASTOS",
                                                           "memory  as QUANTIDADE_AMOSTRAS",
                                                           "ph  as PH",
                                                           "qnt_litros as LITROS_AMOSTRA",
                                                           )

In [36]:
dfRandonTratado.show()


+---+-------------+-------------------+----+--------------+
| ID|LITROS_GASTOS|QUANTIDADE_AMOSTRAS|  PH|LITROS_AMOSTRA|
+---+-------------+-------------------+----+--------------+
|  1|   5000050000|                  0|0.16|        100000|
|  2|  20000100000|                 14|0.26|        200000|
|  3|  45000150000|                 15|0.46|        300000|
|  4|  80000200000|                 54|0.58|        400000|
|  5|     1,25E+11|                 55|0.72|        500000|
|  6|   5000050000|                 83|0.13|        100000|
|  7|  20000100000|                 83|0.37|        200000|
|  8|  45000150000|                 84|0.44|        300000|
|  9|  80000200000|                 37|0.78|        400000|
| 10|     1,25E+11|                 38|0.76|        500000|
| 11|   5000050000|                 38|0.13|        100000|
| 12|  20000100000|                 38|0.26|        200000|
| 13|  45000150000|                 39|0.37|        300000|
| 14|  80000200000|                 34|0

In [44]:
# UPANDO OS DADOS TRATADOS COM O  TRATAMENTO

#dfRandonTratado.coalesce(1).write.option("header", True).csv("s3a://catarina-3sprint-sptech-bucket-tratados/randons", emptyValue='')

#dfRandonTratado.coalesce(1).write.option("header", True).csv("s3a://catarina-3sprint-sptech-bucket-cliente/randon", emptyValue='')



22/06/15 03:11:59 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/15 03:11:59 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
